In [1]:
import torch
import random
random.seed(42)
torch.manual_seed(42)
from transformers import LlamaTokenizer, LlamaForCausalLM 

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

model_path ='openlm-research/open_llama_3b_v2'
tokenizer = LlamaTokenizer.from_pretrained(model_path, legacy=True)
base_model =LlamaForCausalLM.from_pretrained(model_path)


from peft import LoraConfig, PeftModel
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = PeftModel(base_model,lora_config,adapter_name="Shakespeare")
device = torch.device("mps")
model.to(device)

def generate_response(prompt_text, model, tokenizer, max_length=30, num_return_sequences=1):
    input_ids = tokenizer.encode(prompt_text, return_tensors="pt").to(device)
    attention_mask = torch.ones(input_ids.shape, device=input_ids.device)
    
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        attention_mask=attention_mask,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
    )
    
    responses =[]
    for response_id in output_sequences:
        response = tokenizer.decode(response_id, skip_special_tokens=True)
        responses.append(response)
        
    return responses

prompt_text = "Uneasy lies the head that wears a crown."
responses = generate_response(prompt_text, model, tokenizer)
for response in responses:
    print(response)

tensor([1.], device='mps:0')
Uneasy lies the head that wears a crown.
- William Shakespeare
The head of the family is the most important person in the world


In [2]:
# 配置数据集
import os 
import requests
file_name="shakespeare.txt"
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
if not os.path.isfile(file_name):
    data = requests.get(url)
    with open(file_name, 'w') as f:
        f.write(data.text)
from transformers import TextDataset
train_dataset = TextDataset(tokenizer=tokenizer, file_path=file_name, block_size=128)[:256]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [3]:
# training
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="out_put",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_eval_batch_size=32,
    evaluation_strategy='no'
)

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: ws169144. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=320, training_loss=2.709520721435547, metrics={'train_runtime': 1315.7902, 'train_samples_per_second': 1.946, 'train_steps_per_second': 0.243, 'total_flos': 6577270554624000.0, 'train_loss': 2.709520721435547, 'epoch': 10.0})

In [5]:
prompt_text = "Uneasy lies the head that wears a crown."
responses = generate_response(prompt_text, model, tokenizer)
for response in responses:
    print(response)

Uneasy lies the head that wears a crown.

MENENIUS:
I'll not be a fool,



In [6]:
save_path ="merged_fine_tuned_openllama2_3b_shakespeare_m1max"
tokenizer.save_pretrained(save_path)
merged_model = model.merge_and_unload()
merged_model.save_pretrained(save_path)